In [1]:
import time
import pandas as pd
from multiprocessing import Pool
from equity_db.api.mongo_connection import MongoAPI
from equity_db.variables.crsp_variables import CRSPVariables


In [2]:
from equity_db.write.insert_to_db import InsertIntoDB
crsp = pd.read_csv('crsp.csv', nrows=500_000)
crsp.columns = [x.lower() for x in crsp.columns]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# utility function to change types and format df
# inserting function that is paralyzed

In [4]:
data = crsp
data = data.set_index('lpermno')
#data.info()

In [7]:
ins = InsertIntoDB(MongoAPI('test'))
data = ins.prep_data_for_format_and_insert(data, 'crsp', '%Y%m%d')
data.info()

100%|██████████| 7/7 [00:00<00:00, 247.58it/s]


Adjusting static data
Adjusting timeseries data
<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 54594 to 85908
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   gvkey      500000 non-null  category      
 1   linkprim   500000 non-null  category      
 2   liid       500000 non-null  category      
 3   linktype   500000 non-null  category      
 4   lpermco    500000 non-null  category      
 5   linkdt     500000 non-null  datetime64[ns]
 6   linkenddt  500000 non-null  object        
 7   iid        500000 non-null  object        
 8   datadate   500000 non-null  datetime64[ns]
 9   tic        500000 non-null  category      
 10  cusip      500000 non-null  category      
 11  conm       500000 non-null  category      
 12  ajexdi     499950 non-null  float64       
 13  prccd      499950 non-null  float64       
 14  prchd      499950 non-null  float64       
 15  prcld      499950

In [ ]:
variables = CRSPVariables()
static_cols = set(variables.static).intersection(set(data.columns))
timeseries_cols = set(variables.timeseries).intersection(set(data.columns))
#
# for col in static_cols:
#     if variables.get_type(col) == 'double':
#         data[col] = data[col].astype(int).astype(str).astype("category")
#
#     else:
#         data[col] = data[col].astype(str).astype("category")
#
# for col in timeseries_cols:
#     try:
#         data[col] = data[col].astype(float)
#
#     except Exception:
#         print(col)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 54594 to 85908
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   gvkey      500000 non-null  int64  
 1   linkprim   500000 non-null  object 
 2   liid       500000 non-null  int64  
 3   linktype   500000 non-null  object 
 4   lpermco    500000 non-null  int64  
 5   linkdt     500000 non-null  int64  
 6   linkenddt  500000 non-null  object 
 7   iid        500000 non-null  int64  
 8   datadate   500000 non-null  int64  
 9   tic        500000 non-null  object 
 10  cusip      500000 non-null  object 
 11  conm       500000 non-null  object 
 12  ajexdi     499950 non-null  float64
 13  prccd      499950 non-null  float64
 14  prchd      499950 non-null  float64
 15  prcld      499950 non-null  float64
 16  prcod      490447 non-null  float64
 17  exchg      500000 non-null  int64  
 18  tpci       500000 non-null  object 
 19  cik        499139 no

In [ ]:
# paralyze formatting and inserting
# insert in batches
api = MongoAPI('test')


def paralized_format(input_tuple):
    ids, df = input_tuple
    list_of_docs = []
    for asset_id in ids:  # will print progress bar

        # formatting the document data for a single asset at a time
        data_tick = df.loc[asset_id]
        static_df = data_tick.iloc[0].to_frame().reindex(static_cols)

        ticker_dict = list(static_df.to_dict().values())[0]
        ticker_dict['lpermno'] = asset_id
        ticker_dict['timeseries'] = list(data_tick[timeseries_cols].reset_index().to_dict('index').values())
        list_of_docs.append(ticker_dict)

    print('inserting')
    MongoAPI('test').batch_insert('crsp', list_of_docs)
    del list_of_docs
    # gc.collect()


In [ ]:

def chunks(d, chunk_size):
    id_list = sorted(d.index.unique().tolist())
    d = d.sort_index()
    for i in range(0, len(id_list), chunk_size):
        chunk = id_list[i:i + chunk_size]
        yield chunk, d.loc[chunk[0]:chunk[-1]]

#pd.concat([x[1] for x in chunks(data, 100)])

In [ ]:
time2s = time.time()
c = chunks(data, 300)
with Pool() as pool:
        out = pool.map(paralized_format, c, chunksize=1)

print(time.time() - time2s)


In [ ]:
import multiprocessing as mp
import numpy as np
import tqdm
from itertools import repeat
from multiprocessing import Process, Manager
from multiprocessing import Pool
import pandas as pd

num_partitions = mp.cpu_count()
manager = Manager()
d = manager.dict()
c = chunks(data, 300)
pool = Pool(num_partitions)
shared_arg = repeat(d,num_partitions)
pool.map(paralized_format, c)

pool.close()
pool.join()